In [1]:
from tensorflow import keras
import tensorflow
import numpy as np
import pandas as pd 

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [3]:
df = pd.read_csv('customer.csv', na_values  = ' ')

In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [6]:
df['TotalCharges'].fillna(df['TotalCharges'].mean(), inplace = True)

In [7]:
y = np.where(df['Churn'] == 'No', 0,1)
x = df.drop('Churn', axis =1)

In [8]:
x.drop( 'customerID' ,axis =1, inplace = True)

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']

for col in cols:
    x[col]= le.fit_transform(x[col])

In [10]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
cols = ['tenure','MonthlyCharges','TotalCharges'] 
x[cols] = mm.fit_transform(x[cols])

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2)

In [12]:
len(x.columns)

19

### Method 1: Making individual layers

In [13]:
model=Sequential()
model.add(Dense(10,input_shape=(19,),activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [14]:
optimizer=Adam(lr=0.001)
model.compile(optimizer,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                200       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 332
Trainable params: 332
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x_train,y_train,epochs=20)

Epoch 1/20
177/177 [==============================] - 1s 2ms/step - loss: 0.6898 - accuracy: 0.5751
Epoch 2/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4941 - accuracy: 0.7377
Epoch 3/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4371 - accuracy: 0.7814
Epoch 4/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4392 - accuracy: 0.7783
Epoch 5/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4275 - accuracy: 0.7910
Epoch 6/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4270 - accuracy: 0.7903
Epoch 7/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4275 - accuracy: 0.7968
Epoch 8/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4128 - accuracy: 0.8020
Epoch 9/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4100 - accuracy: 0.7972
Epoch 10/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4283 - accuracy: 0.7910

In [16]:
model.evaluate(x_test,y_test)

45/45 [==============================] - 0s 3ms/step - loss: 0.4476 - accuracy: 0.7906


[0.4476383626461029, 0.7906316518783569]

In [17]:
y_pred = model.predict(x_test)

In [18]:
y_pred

array([[0.8767715 , 0.12322846],
       [0.5102511 , 0.48974887],
       [0.87384385, 0.12615614],
       ...,
       [0.17839475, 0.82160527],
       [0.961754  , 0.03824597],
       [0.9597785 , 0.04022154]], dtype=float32)

### Method 2: Making multiple layers at once

In [19]:
#Neural network with hidden layer
model=keras.Sequential([
    keras.layers.Dense(10,input_shape=(19,),activation='relu'),
    keras.layers.Dense(5,activation='relu'),                     
    keras.layers.Dense(2,activation='softmax')       
])

In [20]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [21]:
model.fit(x_train,y_train,epochs=20)

Epoch 1/20
177/177 [==============================] - 1s 2ms/step - loss: 0.6710 - accuracy: 0.7408
Epoch 2/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4756 - accuracy: 0.7482
Epoch 3/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4596 - accuracy: 0.7623
Epoch 4/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4350 - accuracy: 0.7844
Epoch 5/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4379 - accuracy: 0.7819
Epoch 6/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4327 - accuracy: 0.7804
Epoch 7/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4312 - accuracy: 0.7905
Epoch 8/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4290 - accuracy: 0.7861
Epoch 9/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4443 - accuracy: 0.7792
Epoch 10/20
177/177 [==============================] - 0s 2ms/step - loss: 0.4376 - accuracy: 0.7863

In [31]:
model.evaluate(x_test,y_test)

1409/1409 [==============================] - 0s 30us/sample - loss: 0.4278 - acc: 0.7928


[0.42777807607643986, 0.79276085]

In [22]:
y_test

array([0, 0, 0, ..., 1, 0, 0])

In [23]:
y_pred = model.predict(x_test)

In [26]:
pred = []
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
    
    
pred = np.array(pred)

In [40]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print(accuracy_score(pred, y_test))
print(classification_report(pred, y_test))
print(confusion_matrix(pred, y_test))

0.7885024840312278
              precision    recall  f1-score   support

           0       0.89      0.83      0.86      1105
           1       0.51      0.63      0.56       304

    accuracy                           0.79      1409
   macro avg       0.70      0.73      0.71      1409
weighted avg       0.81      0.79      0.80      1409

[[919 186]
 [112 192]]
